## layers

In [14]:
from keras.layers import Conv2D, normalization, ReLU, Dense, Dropout, Flatten, BatchNormalization, AvgPool2D
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras import backend as K

## 创建callback

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

## 将卷积网络的输出结果的最后一层卷积，可视化

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import keras.backend as K

import matplotlib.pyplot as mp
import cv2 as cv
import numpy as np

# 加载以训练好的resnet模型，基于imagenet
model = ResNet50(weights='imagenet')

# 测试图片路径
img_path = r'D:\softfiles\workspace\git\mypython\ml\data\objects\training\airplane\0013.jpg'
# 加载图片为PIL对象
img = image.load_img(img_path, target_size=(224, 224))
# 把pil转成array
x = image.img_to_array(img)
# 增加batch维度
x = np.expand_dims(x, axis=0)
# 处理成模型输入格式
x = preprocess_input(x)


# 模型预测
preds = model.predict(x)
# 获取概率前三的预测结果
print('Predicted:', decode_predictions(preds, top=3)[0])


## 以下内容，将实现可视化预测主要区域。将最终的预测输出，反向梯度传向最后一个conv层，本model即为add_16层，
# 获取最大概率预测的下标
argmax = np.argmax(presd[0])
# 获取最后一层argmax的输出结果
air_plane = model.output[:, argmax]
# 获取最后一个卷积层
last_conv_layer = model.get_layer('add_16')

# 将输出结果梯度传向最后的卷积层,去掉第一维[7, 7, 2048]
grads = K.gradients(air_plane, last_conv_layer.output)[0]
# 求梯度平均
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# 构建新的输入，输出结构
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

# 将预测图片放入到新的结构中
pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(2048):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    
heatmap = np.mean(conv_layer_output_value, axis=-1)

# 归一化处理
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

mp.matshow(heatmap)
mp.show()

img = cv.imread(img_path)
heatmap = cv.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv.applyColorMap(heatmap, cv.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img
cv.imwrite('D://test.jpg', superimposed_img)

## Callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger

early_stopping = EarlyStopping(monitor='acc', patience=1)
checkpoint = ModelCheckpoint('D://test.h5', monitor='val_loss', save_best_only=True)
callbaocks_list = [early_stopping, checkpoint]

## TensorBoard

In [ ]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_feature = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_feature)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_feature, 128, input_length=max_len, name='embd'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=r'D:\softfiles\workspace\data\tensorflow\data\imdb\log',
        # record activation histogram every 1 epoch
        histogram_freq=1,
        # record embedding data every 1 epoch
        embeddings_freq=1
    )
]

history = model.fit(x_train, y_train, epochs=20, batch_size=128,
                   validation_split=0.2, callbacks=callbacks)

# Generative deep learning

In [26]:
import numpy as np

def reweight_distribution(origianl_distribution, temperature=0.5):
    distribution = np.log(origianl_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [27]:
import keras
import numpy as np

nie_path = r'D:\softfiles\workspace\data\tensorflow\data\generate\nietzsche.txt'
with open(nie_path, 'r') as f:
    text = f.read().lower()
print('coupus length: ', len(text))

coupus length:  600893


In [29]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 58
Vectorization...


In [38]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [58]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.01), loss='categorical_crossentropy')


In [60]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen -1)
    generated_text = text[start_index: start_index + maxlen]
    print('---generating with seed:', generated_text)
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('---temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 326s 2ms/step - loss: 1.6430 8s - loss: 1.64 - - ETA: 3
---generating with seed: ustra, came!
     the world now laughs, the grisly veil was 
---temperature: 0.2
ustra, came!
     the world now laughs, the grisly veil was the most is the been the reality, and the man is all the ready the most problem and the the desires of the most have the the desires and man in the the the the same the the have the self-as the man is a most the greates of the contempt of the self-problem and the reality, the the man is the have the conturies the the interpretion of the most is the the the most is a many mankind the the great the 
---temperature: 0.5
ost is the the the most is a many mankind the the great the higher the self-reality the great objest or to ablimit
is the fact in the proble of the command, one have the does that the the have the soul, or any
of the onjohible self-the world and interpretion of the sense of the ready of th

KeyboardInterrupt: 